In [53]:
import os
import cupy as np
import pandas as pd
from PIL import Image

# Load and preprocess dataset
input_folders = {
    0: "/kaggle/input/brain-tumors-256x256/Data/glioma_tumor",
    1: "/kaggle/input/brain-tumors-256x256/Data/meningioma_tumor",
    2: "/kaggle/input/brain-tumors-256x256/Data/normal",
    3: "/kaggle/input/brain-tumors-256x256/Data/pituitary_tumor"
}

output_csv_path = "/kaggle/working/combined_dataset.csv"
data_rows = []
for label, folder_path in input_folders.items():
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".jpg"):
            file_path = os.path.join(folder_path, file_name)
            image = Image.open(file_path).convert("L")
            image = image.resize((256, 256))
            image_array = np.array(image).flatten()
            row = [label] + image_array.tolist()
            data_rows.append(row)

columns = ["label"] + [f"pixel{i}" for i in range(256 * 256)]
dataset = pd.DataFrame(data_rows, columns=columns)
dataset.to_csv(output_csv_path, index=False)
print(f"Dataset saved to {output_csv_path}")





KeyboardInterrupt: 

In [ ]:



data = pd.read_csv('/kaggle/working/combined_dataset.csv')
data = np.array(data)
np.random.shuffle(data)

# Training data
m, n = data.shape
data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n] / 255.

# Small noise for generalization
noise = np.random.randn(*X_train.shape) * 0.01
X_train = X_train + noise

_, m_train = X_train.shape



In [ ]:
def init_params():
    W1 = np.random.randn(1024, 65536) * np.sqrt(1/65536)
    b1 = np.zeros((1024, 1))
    W2 = np.random.randn(512, 1024) * np.sqrt(1/1024)
    b2 = np.zeros((512, 1))
    W3 = np.random.randn(4, 512) * np.sqrt(1/512)
    b3 = np.zeros((4, 1))
    return W1, b1, W2, b2, W3, b3

In [ ]:
# Activation Functions
def ReLU(Z):
    return np.maximum(0.01 * Z, Z)

def softmax(Z):
    expZ = np.exp(Z - np.max(Z, axis=0, keepdims=True))  
    return expZ / np.sum(expZ, axis=0, keepdims=True)

In [ ]:
def forward_prop(X, W1, b1, W2, b2, W3, b3):
    Z1 = np.dot(W1, X) + b1
    A1 = ReLU(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = ReLU(Z2)
    Z3 = np.dot(W3, A2) + b3
    A3 = softmax(Z3)
    return Z1, A1, Z2, A2, Z3, A3

In [ ]:
def one_hot(Y):
    Y = Y.astype(int)
    one_hot_Y = np.zeros((4, Y.size))
    one_hot_Y[Y, np.arange(Y.size)] = 1
    return one_hot_Y

def ReLU_deriv(Z):
    return Z > 0

def backward_prop(Z1, A1, Z2, A2, Z3, A3, W1, W2, W3, X, Y):
    one_hot_Y = one_hot(Y)
    dZ3 = A3 - one_hot_Y
    dW3 = (1 / m_train) * np.dot(dZ3, A2.T)
    db3 = (1 / m_train) * np.sum(dZ3, axis=1, keepdims=True)
    dZ2 = np.dot(W3.T, dZ3) * (Z2 > 0)
    dW2 = (1 / m_train) * np.dot(dZ2, A1.T)
    db2 = (1 / m_train) * np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = np.dot(W2.T, dZ2) * (Z1 > 0)
    dW1 = (1 / m_train) * np.dot(dZ1, X.T)
    db1 = (1 / m_train) * np.sum(dZ1, axis=1, keepdims=True)
    return dW1, db1, dW2, db2, dW3, db3
    

In [54]:
alpha = 0.02

def gradient_descent(X, Y, iterations):
    W1, b1, W2, b2, W3, b3 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2, Z3, A3 = forward_prop(X, W1, b1, W2, b2, W3, b3)
        dW1, db1, dW2, db2, dW3, db3 = backward_prop(Z1, A1, Z2, A2, Z3, A3, W1, W2, W3, X, Y)
        W1 -= alpha * dW1
        b1 -= alpha * db1
        W2 -= alpha * dW2
        b2 -= alpha * db2
        W3 -= alpha * dW3
        b3 -= alpha * db3
        if i % 10 == 0:
            predictions = np.argmax(A3, axis=0)
            accuracy = np.mean(predictions == Y)
            print(f"Iteration {i}, Accuracy: {accuracy:.4f}")


In [55]:
gradient_descent(X_train, Y_train, iterations=1000)

Iteration 0, Accuracy: 0.3087
Iteration 10, Accuracy: 0.3869
Iteration 20, Accuracy: 0.4265
Iteration 30, Accuracy: 0.5043
Iteration 40, Accuracy: 0.4661
Iteration 50, Accuracy: 0.5272
Iteration 60, Accuracy: 0.5391
Iteration 70, Accuracy: 0.6116
Iteration 80, Accuracy: 0.5601
Iteration 90, Accuracy: 0.5100
Iteration 100, Accuracy: 0.5892
Iteration 110, Accuracy: 0.6636
Iteration 120, Accuracy: 0.6403
Iteration 130, Accuracy: 0.6775
Iteration 140, Accuracy: 0.5568
Iteration 150, Accuracy: 0.5544
Iteration 160, Accuracy: 0.7085
Iteration 170, Accuracy: 0.5949
Iteration 180, Accuracy: 0.7471
Iteration 190, Accuracy: 0.6226
Iteration 200, Accuracy: 0.7610
Iteration 210, Accuracy: 0.6760
Iteration 220, Accuracy: 0.7844
Iteration 230, Accuracy: 0.7929
Iteration 240, Accuracy: 0.7295
Iteration 250, Accuracy: 0.8030
Iteration 260, Accuracy: 0.7624
Iteration 270, Accuracy: 0.8068
Iteration 280, Accuracy: 0.7915
Iteration 290, Accuracy: 0.8220
Iteration 300, Accuracy: 0.8325
Iteration 310, Accu